# 글꼴 설정

In [ ]:
import sys
import matplotlib.font_manager as fm


if 'google.colab' in sys.modules:
  !echo 'debconf debconf/frontend select Noninteractive' | \
  debconf-set-selections

  !sudo apt-get -qq -y install fonts-nanum
  fm._rebuild()

Selecting previously unselected package fonts-nanum.
(Reading database ... 129501 files and directories currently installed.)
Preparing to unpack .../fonts-nanum_20180306-3_all.deb ...
Unpacking fonts-nanum (20180306-3) ...
Setting up fonts-nanum (20180306-3) ...
Processing triggers for fontconfig (2.13.1-2ubuntu3) ...


# Import library

In [ ]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import matplotlib.font_manager as fm
fm.findSystemFonts()

['/usr/share/fonts/truetype/liberation/LiberationSans-BoldItalic.ttf',
 '/usr/share/fonts/truetype/liberation/LiberationSansNarrow-BoldItalic.ttf',
 '/usr/share/fonts/truetype/liberation/LiberationSerif-Italic.ttf',
 '/usr/share/fonts/truetype/liberation/LiberationSans-Italic.ttf',
 '/usr/share/fonts/truetype/liberation/LiberationSansNarrow-Regular.ttf',
 '/usr/share/fonts/truetype/liberation/LiberationMono-Bold.ttf',
 '/usr/share/fonts/truetype/liberation/LiberationSansNarrow-Italic.ttf',
 '/usr/share/fonts/truetype/nanum/NanumSquareRoundR.ttf',
 '/usr/share/fonts/truetype/liberation/LiberationSansNarrow-Bold.ttf',
 '/usr/share/fonts/truetype/liberation/LiberationSans-Bold.ttf',
 '/usr/share/fonts/truetype/nanum/NanumMyeongjoBold.ttf',
 '/usr/share/fonts/truetype/nanum/NanumBarunGothicBold.ttf',
 '/usr/share/fonts/truetype/liberation/LiberationSerif-BoldItalic.ttf',
 '/usr/share/fonts/truetype/nanum/NanumSquareR.ttf',
 '/usr/share/fonts/truetype/liberation/LiberationSans-Regular.ttf',

#Basic Setting

In [ ]:
plt.rc('font', family='NanuBarunGothic', size=10)
plt.rcParams['figure.dpi'] = 100

In [ ]:
plt.rcParams['font.family']

['NanuBarunGothic']

In [ ]:
from pandas import read_sql_query as sql_q
from pandas import read_sql_table as sql_t

In [ ]:
conn = sqlite3.connect('/content/drive/MyDrive/Colab Notebooks/instacart.db')
c = conn.cursor()

In [ ]:
c.execute(
    "SELECT name FROM sqlite_master\
    WHERE type IN ('table', 'view') AND name NOT LIKE 'sqlite_%'\
    UNION ALL\
    SELECT name FROM sqlite_temp_master\
    WHERE type IN ('table', 'view')\
    ORDER BY 1"
)

table_list = [table[0] for table in c.fetchall()]
print(table_list)

['aisles', 'departments', 'order_products__prior', 'orders', 'products']


# "instacart" DB - Table List

In [ ]:
for table in table_list:
  _ = sql_q(f"SELECT * FROM {table}",conn)
  print(table)
  print(_.head())
  print()

aisles
   aisle_id                       aisle
0         1       prepared soups salads
1         2           specialty cheeses
2         3         energy granola bars
3         4               instant foods
4         5  marinades meat preparation

departments
   department_id department
0              1     frozen
1              2      other
2              3     bakery
3              4    produce
4              5    alcohol

order_products__prior
   order_id  product_id  add_to_cart_order  reordered
0      2581       40174                  1          0
1      2581       17461                  2          0
2      2581       19731                  3          1
3      5880       24838                  1          1
4      5880       24852                  2          1

orders
   order_id  user_id eval_set  order_number  order_dow order_hour_of_day  \
0   2352815   185282    prior             5          2                09   
1   1414622    41949    prior             9          0           

# 지표 추출 리스트
- 1) 전체 주문 건수
- 2) 구매자 수
- 3) 상품별 주문 건수
- 4) 카트에 가장 먼저 넣는 10개 상품
- 5) 시간별 주문 건수
- 6) 첫 구매 후 다음 구매까지 걸린 평균 일수
- 7) 주문 건당 평균 구매 상품수 UPT(Unit Per Transaction)
- 8) 인당 평균 주문 거수
- 9) 재구매율이 가장 높은 10개의 상품
- 10) Department별 재구매율이 가장 높은 상품 10개

### 1) 전체 주문 건수
### 2) 구매자 수

In [ ]:
# 전체 주문 건수 ; 중복 제거
sql_q(
    "SELECT COUNT(DISTINCT ORDER_ID) F\
    FROM ORDERS;",
    conn
)

,F
0,3220


In [ ]:
# 구매자 수 
sql_q(
    "SELECT COUNT(DISTINCT USER_ID) BU\
    FROM ORDERS;",
    conn
)

,BU
0,3159


### 3) 상품별 주문 건수

In [ ]:
# Join Table
sql_q(
    "SELECT * \
    FROM ORDER_PRODUCTS__PRIOR A\
    LEFT JOIN PRODUCTS B\
    ON A.PRODUCT_ID = B.PRODUCT_ID",
    conn
).head(10)

,order_id,product_id,add_to_cart_order,reordered,product_id,product_name,aisle_id,department_id
0,2581,40174,1,0,40174,2% Reduced Fat Organic Milk,84,16
1,2581,17461,2,0,17461,Air Chilled Organic Boneless Skinless Chicken ...,35,12
2,2581,19731,3,1,19731,Organic Orange Juice With Calcium & Vitamin D,31,7
3,5880,24838,1,1,24838,Unsweetened Almondmilk,91,16
4,5880,24852,2,1,24852,Banana,24,4
5,5880,16797,3,1,16797,Strawberries,24,4
6,5880,5456,4,0,5456,Green Tea With Ginseng and Honey,96,20
7,5880,25199,5,0,25199,Classic Chicken Salad,1,20
8,6030,280,1,0,280,Makeup Remover Cleansing Towelettes,73,11
9,7374,25146,1,0,25146,Original Orange Juice,31,7


In [ ]:
# Result Query
sql_q(
    "SELECT B.PRODUCT_NAME,\
    COUNT(DISTINCT ORDER_ID) F\
    FROM ORDER_PRODUCTS__PRIOR A\
    LEFT JOIN PRODUCTS B\
    ON A.PRODUCT_ID = B.PRODUCT_ID\
    GROUP BY 1",
    conn
).head(10)

,product_name,F
0,#2 Coffee Filters,1
1,0% Fat Black Cherry Greek Yogurt y,1
2,0% Fat Blueberry Greek Yogurt,1
3,0% Fat Free Organic Milk,4
4,0% Fat Organic Greek Vanilla Yogurt,1
5,0% Fat Superfruits Greek Yogurt,1
6,0% Greek Strained Yogurt,6
7,"0% Greek, Blueberry on the Bottom Yogurt",2
8,1 % Lowfat Milk,1
9,1 Apple + 1 Mango Fruit Bar,1


### 4) 장바구니에 가장 먼저 넣는 상품 10개

In [ ]:
# Basic Query
sql_q(
    "SELECT PRODUCT_ID,\
    CASE WHEN ADD_TO_CART_ORDER = 1 THEN 1 ELSE 0 END F_1st\
    FROM ORDER_PRODUCTS__PRIOR;",
    conn
).head(10)

,product_id,F_1st
0,40174,1
1,17461,0
2,19731,0
3,24838,1
4,24852,0
5,16797,0
6,5456,0
7,25199,0
8,280,1
9,25146,1


In [ ]:
# Result Query
sql_q(
    "SELECT *, DENSE_RANK() OVER(ORDER BY F_1st DESC) RNK\
    FROM\
    (SELECT PRODUCT_ID,\
    SUM(CASE WHEN ADD_TO_CART_ORDER = 1 THEN 1 ELSE 0 END) F_1st\
    FROM ORDER_PRODUCTS__PRIOR\
    GROUP BY 1) BASE\
    LIMIT 10;",
    conn
)

,PRODUCT_ID,F_1st,RNK
0,24852,117,1
1,13176,62,2
2,27845,37,3
3,21137,31,4
4,21903,27,5
5,47209,24,6
6,19660,18,7
7,16797,17,8
8,5785,16,9
9,12341,16,9


### 5) 시간별 주문 건수

In [ ]:
# Basic Query
sql_q(
    "SELECT ORDER_HOUR_OF_DAY,ORDER_ID F\
    FROM ORDERS",
    conn
).head(10)

,order_hour_of_day,order_id
0,09,2352815
1,16,1414622
2,13,2534064
3,14,338572
4,13,2578618
5,18,1756647
6,17,2636806
7,15,681993
8,14,826339
9,22,57285


In [ ]:
# Result Query
sql_q(
    "SELECT ORDER_HOUR_OF_DAY,\
    COUNT(DISTINCT ORDER_ID)\
    FROM ORDERS\
    GROUP BY 1\
    ORDER BY 1;",
    conn
).head(10)

,order_hour_of_day,COUNT(DISTINCT ORDER_ID)
0,00,21
1,01,12
2,02,3
3,03,7
4,04,2
5,05,4
6,06,33
7,07,95
8,08,163
9,09,239


### 6) 첫 구매 후 다음 구매까지 걸린 평균 일수

In [ ]:
# Basic Query
sql_q(
    "SELECT * FROM ORDERS",
    conn
).head(10)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2352815,185282,prior,5,2,09,6
1,1414622,41949,prior,9,0,16,6
2,2534064,24631,prior,66,1,13,4
3,338572,95289,prior,4,2,14,18
4,2578618,170417,prior,5,1,13,14
5,1756647,127210,prior,31,3,18,9
6,2636806,26473,prior,24,2,17,17
7,681993,89155,prior,17,1,15,4
8,826339,67750,prior,7,3,14,17
9,57285,89307,prior,48,1,22,5


- order_number : 주문 건수
- days_since_prior_order : 이전 주문 건 이후 다음 주문까지 걸린 일자

In [ ]:
# Result Query
sql_q(
    "SELECT AVG(DAYS_SINCE_PRIOR_ORDER) AVG_RECENCY\
    FROM ORDERS\
    WHERE ORDER_NUMBER = 2 ;",
    conn
)

,AVG_RECENCY
0,13.651786


### 7) 주문 건당 평균 구매 상품 수(UPT, Unit Per Transation)

In [ ]:
# Basci Query
sql_q(
    "SELECT * FROM ORDER_PRODUCTS__PRIOR;",
    conn
).head(10)

,order_id,product_id,add_to_cart_order,reordered
0,2581,40174,1,0
1,2581,17461,2,0
2,2581,19731,3,1
3,5880,24838,1,1
4,5880,24852,2,1
5,5880,16797,3,1
6,5880,5456,4,0
7,5880,25199,5,0
8,6030,280,1,0
9,7374,25146,1,0


In [ ]:
# Result Query
sql_q(
    "SELECT (COUNT(PRODUCT_ID) + 0.0) / COUNT(DISTINCT ORDER_ID) UPT\
    FROM ORDER_PRODUCTS__PRIOR;",
    conn
)

,UPT
0,10.046671


### 8) 인당 평균 주문 건수

In [ ]:
# Result Query
sql_q(
    "SELECT (COUNT(DISTINCT ORDER_ID) + 0.0) / COUNT(DISTINCT USER_ID) AVG_F\
    FROM ORDERS;",
    conn
)

,AVG_F
0,1.01931


### 9) 재구매율이 가장 높은 10개의 상품

In [ ]:
# Basic Query
sql_q(
    "SELECT * FROM ORDER_PRODUCTS__PRIOR\
    LIMIT 10;",
    conn
)

,order_id,product_id,add_to_cart_order,reordered
0,2581,40174,1,0
1,2581,17461,2,0
2,2581,19731,3,1
3,5880,24838,1,1
4,5880,24852,2,1
5,5880,16797,3,1
6,5880,5456,4,0
7,5880,25199,5,0
8,6030,280,1,0
9,7374,25146,1,0


In [ ]:
# Result Query
sql_q(
    "SELECT PRODUCT_ID, RET / TOTAL RET_RATIO\
    FROM\
    (SELECT PRODUCT_ID,\
    (SUM(CASE WHEN REORDERED = 1 THEN 1 ELSE 0 END) + 0.0) RET,\
    COUNT(*) TOTAL\
    FROM ORDER_PRODUCTS__PRIOR\
    GROUP BY 1\
    ORDER BY 2 DESC) BASE\
    ORDER BY  2 DESC;",
    conn
).head(10)

,PRODUCT_ID,RET_RATIO
0,20574,1.0
1,44765,1.0
2,12384,1.0
3,47156,1.0
4,6104,1.0
5,27730,1.0
6,11123,1.0
7,31371,1.0
8,21386,1.0
9,30639,1.0


### 10) Department별 재구매율이 가장 높은 상품 10개

In [ ]:
# JOIN Table
sql_q(
    "SELECT C.DEPARTMENT, A.PRODUCT_ID,\
    SUM(REORDERED) / COUNT(*) RET_RATIO\
    FROM ORDER_PRODUCTS__PRIOR A\
    LEFT JOIN PRODUCTS B\
    ON A.PRODUCT_ID = B.PRODUCT_ID\
    LEFT JOIN DEPARTMENTS C\
    ON B.DEPARTMENT_ID = C.DEPARTMENT_ID\
    GROUP BY 1,2\
    ORDER BY 3 DESC, 2 ASC;",conn
).head(10)

,department,product_id,RET_RATIO
0,meat seafood,23,1
1,snacks,25,1
2,canned goods,29,1
3,snacks,32,1
4,canned goods,37,1
5,meat seafood,40,1
6,international,47,1
7,bakery,101,1
8,babies,106,1
9,frozen,128,1


# 구매자 분석
- 10분위 분석

In [ ]:
# Basic Query 1
sql_q(
    "SELECT * FROM ORDERS;",
    conn
).head(10)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2352815,185282,prior,5,2,09,6
1,1414622,41949,prior,9,0,16,6
2,2534064,24631,prior,66,1,13,4
3,338572,95289,prior,4,2,14,18
4,2578618,170417,prior,5,1,13,14
5,1756647,127210,prior,31,3,18,9
6,2636806,26473,prior,24,2,17,17
7,681993,89155,prior,17,1,15,4
8,826339,67750,prior,7,3,14,17
9,57285,89307,prior,48,1,22,5


In [ ]:
# Basic Query 2
sql_q(
    "SELECT *,\
    ROW_NUMBER() OVER(ORDER BY F DESC) RNK\
    FROM\
    (SELECT USER_ID, COUNT(DISTINCT ORDER_ID) F\
    FROM ORDERS\
    GROUP BY 1) BASE",
    conn
).head(10)

,USER_ID,F,RNK
0,32099,3,1
1,2610,2,2
2,10132,2,3
3,10972,2,4
4,12435,2,5
5,22379,2,6
6,25449,2,7
7,33244,2,8
8,37499,2,9
9,37801,2,10


In [ ]:
# 분위수 확인
sql_q(
    "SELECT COUNT(DISTINCT USER_ID)\
    FROM ORDERS;",
    conn
)

,COUNT(DISTINCT USER_ID)
0,3159


In [ ]:
# Process Query
sql_q(
    "SELECT *,\
    CASE\
    WHEN RNK <= 316 THEN 'Q_1'\
    WHEN RNK <= 632 THEN 'Q_2'\
    WHEN RNK <= 948 THEN 'Q_3'\
    WHEN RNK <= 1264 THEN 'Q_4'\
    WHEN RNK <= 1580 THEN 'Q_5'\
    WHEN RNK <= 1895 THEN 'Q_6'\
    WHEN RNK <= 2211 THEN 'Q_7'\
    WHEN RNK <= 2527 THEN 'Q_8'\
    WHEN RNK <= 2843 THEN 'Q_9'\
    WHEN RNK <= 3159 THEN 'Q_10' END Quantile\
    FROM\
    (SELECT *,\
    ROW_NUMBER() OVER(ORDER BY F DESC) RNK\
    FROM\
    (SELECT USER_ID, COUNT(DISTINCT ORDER_ID) F\
    FROM ORDERS\
    GROUP BY 1) BASE) BASE2",
    conn
).head(10)

,USER_ID,F,RNK,Quantile
0,32099,3,1,Q_1
1,2610,2,2,Q_1
2,10132,2,3,Q_1
3,10972,2,4,Q_1
4,12435,2,5,Q_1
5,22379,2,6,Q_1
6,25449,2,7,Q_1
7,33244,2,8,Q_1
8,37499,2,9,Q_1
9,37801,2,10,Q_1


In [ ]:
# 참조 Table 생성
c.execute(
    "CREATE TABLE IF NOT EXISTS USER_QUANTILE AS\
    SELECT *,\
    CASE\
    WHEN RNK <= 316 THEN 'Q_1'\
    WHEN RNK <= 632 THEN 'Q_2'\
    WHEN RNK <= 948 THEN 'Q_3'\
    WHEN RNK <= 1264 THEN 'Q_4'\
    WHEN RNK <= 1580 THEN 'Q_5'\
    WHEN RNK <= 1895 THEN 'Q_6'\
    WHEN RNK <= 2211 THEN 'Q_7'\
    WHEN RNK <= 2527 THEN 'Q_8'\
    WHEN RNK <= 2843 THEN 'Q_9'\
    WHEN RNK <= 3159 THEN 'Q_10' END Quantile\
    FROM\
    (SELECT *,\
    ROW_NUMBER() OVER(ORDER BY F DESC) RNK\
    FROM\
    (SELECT USER_ID, COUNT(DISTINCT ORDER_ID) F\
    FROM ORDERS\
    GROUP BY 1) BASE) BASE2;"
)

sql_q(
    "SELECT * FROM USER_QUANTILE;",
    conn
).head(10)

,USER_ID,F,RNK,Quantile
0,32099,3,1,Q_1
1,2610,2,2,Q_1
2,10132,2,3,Q_1
3,10972,2,4,Q_1
4,12435,2,5,Q_1
5,22379,2,6,Q_1
6,25449,2,7,Q_1
7,33244,2,8,Q_1
8,37499,2,9,Q_1
9,37801,2,10,Q_1


In [ ]:
# 각 분위별 전체 주문건수 특성
sql_q(
    "SELECT QUANTILE, SUM(F) F\
    FROM USER_QUANTILE\
    GROUP BY 1\
    ORDER BY QUANTILE;",
    conn
).head(10)

,Quantile,F
0,Q_1,377
1,Q_10,316
2,Q_2,316
3,Q_3,316
4,Q_4,316
5,Q_5,316
6,Q_6,315
7,Q_7,316
8,Q_8,316
9,Q_9,316


- 분위별 전체 주문 건수가 큰 차이 없이 고르게 분포 되어있음

# 상품 분석
- 재구매 비중과 주문 건수(재주문 10건 이상) 확인

In [ ]:
# Basic Query
sql_q(
    "SELECT PRODUCT_ID, (SUM(REORDERED) + 0.0) / SUM(1) REORDER_RATE,\
    COUNT(DISTINCT ORDER_ID) F\
    FROM ORDER_PRODUCTS__PRIOR\
    GROUP BY 1",
    conn
).head(10)

,product_id,REORDER_RATE,F
0,1,0.333333,3
1,10,0.500000,2
2,11,0.000000,1
3,23,1.000000,1
4,25,1.000000,1
5,29,1.000000,1
6,32,1.000000,1
7,34,0.333333,9
8,35,0.000000,1
9,37,1.000000,1


In [ ]:
# JOIN TABLE
sql_q(
    "SELECT * FROM ORDER_PRODUCTS__PRIOR A\
    LEFT JOIN PRODUCTS B\
    ON A.PRODUCT_ID = B.PRODUCT_ID",
    conn
)

,order_id,product_id,add_to_cart_order,reordered,product_id,product_name,aisle_id,department_id
0,2581,40174,1,0,40174,2% Reduced Fat Organic Milk,84,16
1,2581,17461,2,0,17461,Air Chilled Organic Boneless Skinless Chicken ...,35,12
2,2581,19731,3,1,19731,Organic Orange Juice With Calcium & Vitamin D,31,7
3,5880,24838,1,1,24838,Unsweetened Almondmilk,91,16
4,5880,24852,2,1,24852,Banana,24,4
...,...,...,...,...,...,...,...,...
32285,3420098,46676,9,0,46676,Total 0% Nonfat Greek Yogurt,120,16
32286,3420098,5240,10,0,5240,Spaghetti No 12,131,9
32287,3420098,23375,11,0,23375,Marinara Sauce,9,9
32288,3420098,42342,12,0,42342,Roasted Turkey Breast,96,20


In [ ]:
# Result Query
sql_q(
    "SELECT A.PRODUCT_ID, B.PRODUCT_NAME, DEPARTMENT_ID,\
    ( SUM(REORDERED) + 0.0 ) / SUM(1) REORDER_RATE,\
    COUNT(DISTINCT ORDER_ID) F\
    FROM ORDER_PRODUCTS__PRIOR A\
    LEFT JOIN PRODUCTS B\
    ON A.PRODUCT_ID = B.PRODUCT_ID\
    GROUP BY A.PRODUCT_ID\
    HAVING F >= 10\
    ORDER BY 4 DESC;",
    conn
).head(10)

,product_id,product_name,department_id,REORDER_RATE,F
0,44765,Colby Cheese Sticks,16,1.000000,12
1,20574,Roasted Turkey,20,1.000000,13
2,12384,Organic Lactose Free 1% Lowfat Milk,16,1.000000,11
3,28465,Icelandic Style Skyr Blueberry Non-fat Yogurt,16,0.958333,24
4,38164,Almonds & Sea Salt in Dark Chocolate,19,0.933333,15
5,29447,"Milk, Organic, Vitamin D",16,0.933333,15
6,47788,Honeydew Melon,4,0.923077,13
7,20082,Organic Whole Milk with DHA Omega-3,16,0.923077,13
8,1511,2% Reduced Fat DHA Omega-3 Reduced Fat Milk,16,0.923077,13
9,46616,Organic Dark Chocolate Peanut Butter Cups,19,0.916667,12


- 10건 이상의 재주문을 보이는 상품 중 유제품 분야의 상품이 재주문율이 높은 것을 확인할 수 있음

### 다음 구매까지의 소요 기간과 재구매 관계
- 10분위 분석 : 재구매율이 높은 순서대로 상품을 10가지 그룹으로 구분
- 가정 : 재구매율이 높은 상품은 일정한 주기를 가질 것이다.

In [ ]:
# 참조 VIEW 생성

c.execute("CREATE VIEW IF NOT EXISTS USER_REORDER AS\
    SELECT A.PRODUCT_ID, B.PRODUCT_NAME, DEPARTMENT_ID,\
    ( SUM(REORDERED) + 0.0 ) / SUM(1) REORDER_RATE,\
    COUNT(DISTINCT ORDER_ID) F\
    FROM ORDER_PRODUCTS__PRIOR A\
    LEFT JOIN PRODUCTS B\
    ON A.PRODUCT_ID = B.PRODUCT_ID\
    GROUP BY A.PRODUCT_ID\
    HAVING F > 10\
    ORDER BY 4 DESC;")

sql_q(
    "SELECT * FROM USER_REORDER;",
    conn
).head(10)

,product_id,product_name,department_id,REORDER_RATE,F
0,44765,Colby Cheese Sticks,16,1.000000,12
1,20574,Roasted Turkey,20,1.000000,13
2,12384,Organic Lactose Free 1% Lowfat Milk,16,1.000000,11
3,28465,Icelandic Style Skyr Blueberry Non-fat Yogurt,16,0.958333,24
4,38164,Almonds & Sea Salt in Dark Chocolate,19,0.933333,15
5,29447,"Milk, Organic, Vitamin D",16,0.933333,15
6,47788,Honeydew Melon,4,0.923077,13
7,20082,Organic Whole Milk with DHA Omega-3,16,0.923077,13
8,1511,2% Reduced Fat DHA Omega-3 Reduced Fat Milk,16,0.923077,13
9,46616,Organic Dark Chocolate Peanut Butter Cups,19,0.916667,12


In [ ]:
# Process Query
sql_q(
    "SELECT *, ROW_NUMBER() OVER(ORDER BY REORDER_RATE DESC) RNK\
    FROM USER_REORDER;",
    conn
).head(10)

,product_id,product_name,department_id,REORDER_RATE,F,RNK
0,44765,Colby Cheese Sticks,16,1.000000,12,1
1,20574,Roasted Turkey,20,1.000000,13,2
2,12384,Organic Lactose Free 1% Lowfat Milk,16,1.000000,11,3
3,28465,Icelandic Style Skyr Blueberry Non-fat Yogurt,16,0.958333,24,4
4,38164,Almonds & Sea Salt in Dark Chocolate,19,0.933333,15,5
5,29447,"Milk, Organic, Vitamin D",16,0.933333,15,6
6,47788,Honeydew Melon,4,0.923077,13,7
7,20082,Organic Whole Milk with DHA Omega-3,16,0.923077,13,8
8,1511,2% Reduced Fat DHA Omega-3 Reduced Fat Milk,16,0.923077,13,9
9,46616,Organic Dark Chocolate Peanut Butter Cups,19,0.916667,12,10


In [ ]:
sql_q(
    "SELECT COUNT(*) FROM USER_REORDER;",
    conn
)

,COUNT(*)
0,486


In [ ]:
sql_q(
    "SELECT PRODUCT_ID, DEPARTMENT_ID,\
    CASE\
    WHEN RNK <= 54 THEN 'Q_1'\
    WHEN RNK <= 102 THEN 'Q_2'\
    WHEN RNK <= 150 THEN 'Q_3'\
    WHEN RNK <= 198 THEN 'Q_4'\
    WHEN RNK <= 246 THEN 'Q_5'\
    WHEN RNK <= 294 THEN 'Q_6'\
    WHEN RNK <= 342 THEN 'Q_7'\
    WHEN RNK <= 390 THEN 'Q_8'\
    WHEN RNK <= 438 THEN 'Q_9'\
    WHEN RNK <= 486 THEN 'Q_10' END RNK_GRP\
    FROM\
    (SELECT *, ROW_NUMBER() OVER(ORDER BY REORDER_RATE DESC) RNK\
    FROM USER_REORDER) BASE;",
    conn
).head(11)

,product_id,department_id,RNK_GRP
0,44765,16,Q_1
1,20574,20,Q_1
2,12384,16,Q_1
3,28465,16,Q_1
4,38164,19,Q_1
5,29447,16,Q_1
6,47788,4,Q_1
7,20082,16,Q_1
8,1511,16,Q_1
9,46616,19,Q_1


In [ ]:
# 참조 VIEW 생성
c.execute(
    "CREATE VIEW IF NOT EXISTS TEMP_PRIOR AS\
    SELECT PRODUCT_ID, DAYS_SINCE_PRIOR_ORDER\
    FROM ORDER_PRODUCTS__PRIOR A\
    INNER JOIN ORDERS B\
    ON A.ORDER_ID = B.ORDER_ID;"
)

In [ ]:
c.execute(
    "CREATE VIEW IF NOT EXISTS USER_REORDER2 AS\
    SELECT PRODUCT_ID, DEPARTMENT_ID,\
    CASE\
    WHEN RNK <= 54 THEN 'Q_1'\
    WHEN RNK <= 102 THEN 'Q_2'\
    WHEN RNK <= 150 THEN 'Q_3'\
    WHEN RNK <= 198 THEN 'Q_4'\
    WHEN RNK <= 246 THEN 'Q_5'\
    WHEN RNK <= 294 THEN 'Q_6'\
    WHEN RNK <= 342 THEN 'Q_7'\
    WHEN RNK <= 390 THEN 'Q_8'\
    WHEN RNK <= 438 THEN 'Q_9'\
    WHEN RNK <= 486 THEN 'Q_10' END RNK_GRP\
    FROM\
    (SELECT *, ROW_NUMBER() OVER(ORDER BY REORDER_RATE DESC) RNK\
    FROM USER_REORDER) BASE;"
)

In [ ]:
# Result Query

sql_q(
    "SELECT RNK_GRP, AVG(VAR_DAYS) AVG_VAR_DAYS\
    FROM \
    (SELECT A.RNK_GRP, A.PRODUCT_ID,\
    SUM((DAYS_SINCE_PRIOR_ORDER-(SELECT AVG(DAYS_SINCE_PRIOR_ORDER) FROM TEMP_PRIOR))*\
           (DAYS_SINCE_PRIOR_ORDER-(SELECT AVG(DAYS_SINCE_PRIOR_ORDER) FROM TEMP_PRIOR)) ) / (COUNT(DAYS_SINCE_PRIOR_ORDER)-1) VAR_DAYS\
    FROM USER_REORDER2 A\
    LEFT JOIN TEMP_PRIOR B\
    ON A.PRODUCT_ID = B.PRODUCT_ID\
    GROUP BY 1,2) BASE\
    GROUP BY 1\
    ORDER BY 2;",
    conn
)

,RNK_GRP,AVG_VAR_DAYS
0,Q_1,69.086900
1,Q_2,70.292588
2,Q_6,75.983647
3,Q_4,78.161848
4,Q_3,78.520822
5,Q_5,78.758276
6,Q_10,82.209698
7,Q_7,82.700302
8,Q_9,83.857108
9,Q_8,83.985559


- 분석 결과로 재구매율에 따른 각 분위에서 재구매소요 기간에 대한 분산을 보았을 때 10건 이상의 재구매가 있는 상품 중에서 재구매율이 높을수록 유의미한 주기의 차이가 있는 것을 확인할 수 있음.